In [14]:
# Standard libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
import librosa
from google.colab import drive
import os

from librosa import load as lload
from librosa.feature import mfcc
from librosa.feature.inverse import mfcc_to_audio

# Mount Google Drive for access
drive.mount('/content/drive')

from sklearn.preprocessing import MinMaxScaler


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:

# Function to extract 5-second  chunks from audio
def extract_5sec_chunks(
    audio_array: np.ndarray,
    window_size_s: float = 5.0,
    hop_size_s: float = 2.5, #(with overlap)
    sample_rate=16000,
    # n_fft=2048,
    # n_mfcc=13,
) -> np.ndarray:
    frame_length = int(window_size_s * sample_rate)
    frame_step = int(hop_size_s * sample_rate)
    framed_audio = tf.signal.frame(audio_array, frame_length, frame_step, pad_end=False)

    return framed_audio



In [16]:


# Load metadata
dataset_path = '/content/drive/MyDrive/207-Project/Inference/'
metadata_path = os.path.join(dataset_path, "test.csv")
test_df = pd.read_csv(metadata_path)
# Create a dictionary to map filenames to labels
labels_dict = test_df.set_index('filename_npy')['primary_label'].to_dict()

# Get unique filenames from the metadata
base_directory_path = '/content/drive/MyDrive/207-Project/notebooks/RG/3_species/librosa_loaded_sr16000/'
filenames = set(base_directory_path + '/' + test_df['filename_npy'].unique())


In [17]:
def mfcc_chunks(dataframe: pd.DataFrame, sample_rate=16000, n_mfcc=20) -> tuple[np.ndarray, np.ndarray]:
    y = []
    X = []

    for i, row in dataframe.iterrows():
      label = row['primary_label']
      tensor_5sec = row['audio_chunks']

      for each in tensor_5sec:
        y.append(label)
        each = np.array(each)
        X_mfcc = mfcc(y=each, sr=sample_rate)
        X.append(X_mfcc)

    assert len(y) == len(X)

    y = np.array(y)
    X = np.array(X)



    return X, y

In [18]:
# Load all test audio data one time
test_audios = []
for filename in test_df['filename_npy']:
    audio = np.load('/content/drive/MyDrive/207-Project/data/test/librosa_loaded/' + filename)
    test_audios.append(audio)

In [19]:
print(len(test_audios))


381


In [20]:
#Initialize an empty list to store the chunk data for test data
test_chunks = []

for audio in test_audios:
  test_chunks.append(extract_5sec_chunks(audio))

len(test_chunks)

381

In [21]:
#add chunks to test data
test_df['audio_chunks'] = test_chunks
test_df.head()
test_df = test_df.sample(frac=1, random_state=1234)

In [12]:
del audio
del base_directory_path
del dataset_path
del drive
del extract_5sec_chunks
del filename
del filenames
# del labels_dict
del layers
del metadata_path
del os
del test_audios
del test_chunks

In [22]:
#convert to X and y
X_test, y_test = mfcc_chunks(test_df)

In [23]:
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_test shape: (5786, 20, 157)
y_test shape: (5786,)


In [24]:
# validate_df.head()
print(test_df['primary_label'].unique())

['barswa' 'comsan' 'eaywag1']


In [25]:
del test_df
del mfcc
del mfcc_chunks
del mfcc_to_audio

In [26]:
import json

In [28]:
X_test_list = X_test.tolist()
y_test_list = y_test.tolist()



# Save data to JSON files
with open('/content/drive/MyDrive/My-207/X_test_mfcc.json', 'w') as file:
    json.dump(X_test_list, file)

with open('/content/drive/MyDrive/My-207/y_test_mfcc.json', 'w') as file:
    json.dump(y_test_list, file)

